In [155]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import numpy as np
from crepes import WrapClassifier
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import log_loss, brier_score_loss, accuracy_score, roc_auc_score
from sklearn.calibration import calibration_curve
from typing import List
from venn_abers import VennAbers
from sklearn.utils.validation import check_is_fitted

In [156]:
# Gerar um conjunto de dados de classificação
X, y = make_classification(n_samples=100000, n_features=4)

In [157]:
# Dividir os dados em conjuntos de treinamento, calibração e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_calib, y_train, y_calib = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [158]:
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [159]:
# Size of calibration data
n = len(X_test)
# Get the probability predictions
y_prob = rf.predict_proba(X_test)
# We only need the probability for the true class
prob_true_class = y_prob[np.arange(n),y_test]
# Turn into uncertainty score (larger means more uncertain)
nc_score = 1 - prob_true_class
# Setting the alpha so that we get 95% prediction sets
alpha = 0.05
# define quantile
q_level = np.ceil((n+1)*(1-alpha))/n
qhat = np.quantile(nc_score, q_level, method='higher')
# Get the "probabilities" from the model
y_prob = rf.predict_proba(X_calib)
# Get for each instance the actual probability of ground truth
prob_for_true_class = y_prob[np.arange(len(y_calib)),y_calib]
nc_score = 1 - prob_for_true_class

In [160]:
fig = px.histogram(nc_score, nbins=30, range_x=(0, 1), title="Histograma de Escora de NCScore")
fig.update_layout(hovermode="x")
fig.update_traces(hovertemplate="%{y}")
fig.update_yaxes(title_text="Frequência")
fig.update_xaxes(title_text="Escore de Não Conformidade")
fig.update_layout(showlegend=False)

Utilizando técnico da OOB Sample para Modelo Ensemble

In [161]:
rf = RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=42)
clf = WrapClassifier(rf)
clf.fit(X_train, y_train)
clf.calibrate(X_train, y_train, oob=True)

WrapClassifier(learner=RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=42), calibrated=True, predictor=ConformalClassifier(fitted=True, mondrian=False))

In [162]:
class WrapEnsembleClassifier(VennAbers):

    def __init__(self, learner: RandomForestClassifier):
        check_is_fitted(learner)
        self.learner = learner

    def calibrate(self, y):
        self.fit(self.learner.oob_decision_function_, y)
        return self

    def predict_proba(self, X):
        y_score = self.learner.predict_proba(X)
        p_prime, _ = super().predict_proba(y_score)
        return p_prime

    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

Avaliar o melhor nível de confiança com menor taxa de erro

In [163]:
def get_error_metrics(clf: WrapClassifier, X_test: np.ndarray) -> List:
    error_rate = {k: {} for k in [0.45, 0.40, 0.35, 0.30, 0.25, 0.20, 0.15, 0.10, 0.05]}
    for error in error_rate:
        predict_set = clf.predict_set(X_test, confidence = 1 - error)
        error_rate[error]["efficiency"] = np.sum([np.sum(p) == 1 for p in predict_set]) / len(predict_set)
        error_rate[error]["validity"] = np.sum(predict_set) / len(predict_set)
    return error_rate

In [164]:
error_rate = get_error_metrics(clf, X_test)

In [165]:
def custom_legend(fig, nameSwap):
        for i, dat in enumerate(fig.data):
            for elem in dat:
                if elem == "name":
                    fig.data[i].name = nameSwap[fig.data[i].name]
        return fig

In [166]:
def custom_facet_title(fig, titles):
    for i, label in enumerate(titles):
        fig.layout.annotations[i]['text'] = label
    return fig

In [167]:
# Create a DataFrame from the data

df = pd.DataFrame(error_rate).T

# Create the bar chart
fig = px.line(df, x=df.index, y=['efficiency', 'validity'],
             labels={'value': 'Escore'},
             markers=True,
             title='Relação Eficiência & Solidez versus Taxa de Erro',
             color_discrete_sequence=['darkblue', 'orange'],
             width=800,
             height=400)
fig.update_layout(hovermode="x")
fig.update_traces(hovertemplate="%{y}")
custom_legend(
        fig=fig, nameSwap={"efficiency": "Eficiência", "validity": "Solidez"}
    )
fig.update_layout(legend=dict(
    title="Métrica"))
fig.update_yaxes(title_text="Escore")
fig.update_xaxes(title_text="Taxa de Erro")

Melhor taxa de erro com a maior eficiência

In [168]:
clf.error_rate = df["efficiency"].idxmax()

In [169]:
"""
A lower validity (AvgC) value signifies that the model is better at producing more specific and informative
predictions. 

A higher efficiency (OneC) value indicates that the conformal prediction model produces specific and informative
predictions more efficiently. 

Researchers have determined that the most effective approach is to use a margin-based nonconformity
function to achieve a high rate of singleton predictions (OneC).

"""


'\nA lower validity (AvgC) value signifies that the model is better at producing more specific and informative\npredictions. \n\nA higher efficiency (OneC) value indicates that the conformal prediction model produces specific and informative\npredictions more efficiently. \n\nResearchers have determined that the most effective approach is to use a margin-based nonconformity\nfunction to achieve a high rate of singleton predictions (OneC).\n\n'

In [170]:
#Outputs
#Não há confiança para nenhum dos labels. [0, 0]
#Há confiança de apenas um dos labels. [0, 1]; [1, 0]
#Há confiança em ambos os labels. [1, 1]; [1, 1]

In [171]:
y_prob = clf.predict_set(X_test, confidence=1 - clf.error_rate)[:,1]

In [172]:
accuracy_score(y_test, y_prob)

0.9755

In [173]:
y_prob = clf.predict_proba(X_test)
y_prob = y_prob[:, 1]
v_prob_true, v_prob_pred = calibration_curve(y_test, y_prob, n_bins=10)
l_loss = log_loss(y_test, y_prob)
brier_loss = brier_score_loss(y_test, y_prob)

fig = go.Figure()

# Add traces for each model

fig.add_trace(go.Scatter(x=v_prob_pred, y=v_prob_true, mode='lines+markers', name="RandomForest"))

# Add a trace for the perfectly calibrated line
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Perfectly calibrated', line=dict(dash='dash', color='grey')))

fig.update_layout(
    title="Calibration Plot",
    xaxis_title="Mean predicted probability",
    yaxis_title="Fraction of positives",
    legend_title="Modelos",
    autosize=False,
)

fig.show()

In [174]:
class WrapperOOBConformalClassifier(VennAbers):

    def __init__(self, learner: RandomForestClassifier):
        check_is_fitted(learner)
        self.learner = learner
        self.feature_importances_ = self.learner.feature_importances_

    def calibrate(self, y):
        self.fit(self.learner.oob_decision_function_, y)
        return self

    def predict_proba(self, X):
        y_score = self.learner.predict_proba(X)
        p_prime, _ = super().predict_proba(y_score)
        return p_prime

    def predict(self, X):
        return (self.predict_proba(X)[:, 1] >= 0.50).astype(int)

In [175]:
rf = RandomForestClassifier(n_jobs=-1, random_state=42, oob_score=True)
rf = rf.fit(X_train, y_train)

In [176]:
clf = WrapEnsembleClassifier(rf)
clf.calibrate(y_train)

/home/lucasleao/miniconda3/envs/autorizai/lib/python3.10/site-packages/venn_abers/venn_abers.py:102: RuntimeWarning:

All-NaN slice encountered



In [177]:
p_prime = clf.predict_proba(X_test)
p_prime = p_prime[:, 1]
v_prob_true, v_prob_pred = calibration_curve(y_test, p_prime, n_bins=10)
l_loss = log_loss(y_test, p_prime)
brier_loss = brier_score_loss(y_test, p_prime)

fig = go.Figure()

# Add traces for each model

fig.add_trace(go.Scatter(x=v_prob_pred, y=v_prob_true, mode='lines+markers', name="RandomForest"))

# Add a trace for the perfectly calibrated line
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Perfectly calibrated', line=dict(dash='dash', color='grey')))

fig.update_layout(
    title="Calibration Plot",
    xaxis_title="Mean predicted probability",
    yaxis_title="Fraction of positives",
    legend_title="Modelos",
    autosize=False,
)

fig.show()

E se... calibrarmos via Venn-Abers, e utilizarmos um modelo mondrian com classe condicional?

Fazendo NC Pessoalmente

In [178]:
class WrapperOOBConformalClassifier():

    def __init__(self, learner: RandomForestClassifier):
        check_is_fitted(learner)
        self.learner = learner
        self.calibration_layer = VennAbers()
        self.feature_importances_ = self.learner.feature_importances_
        self.alpha = None
        self.hinge = None
        self.size = len(self.learner.oob_decision_function_)

    def fit(self, y):
        # Get the probability predictions
        y_prob = self.learner.oob_decision_function_

        self.calibration_layer.fit(y_prob, y)

        # We only need the probability for the true class
        prob_true_class = y_prob[np.arange(self.size),y]
        
        # Turn into uncertainty score
        self.hinge = 1 - prob_true_class

        return self
    
    def predict_proba(self, X):
        y_score = self.learner.predict_proba(X)
        p_prime, _ = self.calibration_layer.predict_proba(y_score)
        return p_prime

    def predict(self, X):
        return (self.predict_proba(X)[:, 1] >= 0.50).astype(int)
    
    def predict_set(self, X, confidence=0.95):
        score = self.predict_proba(X)
        nc_score = 1 - score
        error_rate = 1 - confidence
        q_level = np.ceil((self.size+1)*(1-error_rate))/self.size
        qhat = np.quantile(self.hinge, q_level, method='higher')

        return (nc_score <= qhat).astype(int)

In [179]:
clf = WrapperOOBConformalClassifier(rf)

In [180]:
clf.fit(y_train)

/home/lucasleao/miniconda3/envs/autorizai/lib/python3.10/site-packages/venn_abers/venn_abers.py:102: RuntimeWarning:

All-NaN slice encountered



In [181]:
clf.predict_set(X_test)

array([[0, 1],
       [0, 1],
       [0, 1],
       ...,
       [0, 1],
       [1, 0],
       [1, 0]])

In [182]:
error_rate = get_error_metrics(clf, X_test)

In [183]:
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

In [188]:
class WrapperOOBConformalClassifier():
    """
    Esta é uma classe de classificador conformal baseada em Out-of-Bag (OOB) que usa um classificador de floresta aleatória como aprendiz. 
    Inspirada nas classes de WrapperClassifier da biblioteca Crepes.
    """

    def __init__(self, learner: RandomForestClassifier):

        """
        Inicializa o classificador com um aprendiz e uma camada de calibração Venn-Abers.
        """

        check_is_fitted(learner)
        self.learner = learner
        self.calibration_layer = VennAbers()
        self.feature_importances_ = self.learner.feature_importances_
        self.qhat = None

    def fit(self, y):
        """
        Ajusta o classificador aos dados de treinamento. Calcula a pontuação de conformidade para cada instância de treinamento.
        """

        # Get the probability predictions
        y_prob = self.learner.oob_decision_function_

        self.calibration_layer.fit(y_prob, y)

        # We only need the probability for the true class
        n = len(self.learner.oob_decision_function_)
        prob_true_class = y_prob[np.arange(n),y]
        
        # Turn into ncscore, alpha being the error rate (Confidence level of 0.95)
        hinge = 1 - prob_true_class
        alpha = 0.05
        

        q_level = np.ceil((n+1)*(1-alpha))/n
        self.qhat = np.quantile(hinge, q_level, method='higher')

        return self
    
    def predict_proba(self, X):
        """
        Prevê as probabilidades das classes para as instâncias em X.
        """

        y_score = self.learner.predict_proba(X)
        p_prime, _ = self.calibration_layer.predict_proba(y_score)
        return p_prime

    def predict(self, X):
        """
        Prevê as classes para as instâncias em X.
        """

        return (self.predict_proba(X)[:, 1] >= 0.50).astype(int)
    
    def predict_set(self, X):
        """
        Prevê o conjunto de classes possíveis para as instâncias em X com base no nível de significância pré-definido.
        """
        
        score = self.predict_proba(X)
        hinge = 1 - score
        return (hinge <= self.qhat).astype(int)
    
    def expected_calibration_error(self, y, y_prob, M=5):
        # uniform binning approach with M number of bins
        bin_boundaries = np.linspace(0, 1, M + 1)
        bin_lowers = bin_boundaries[:-1]
        bin_uppers = bin_boundaries[1:]

        # get max probability per sample i
        confidences = np.max(y_prob, axis=1)
        # get predictions from confidences (positional in this case)
        predicted_label = np.argmax(y_prob, axis=1)

        # get a boolean list of correct/false predictions
        predictions = predicted_label==y

        ece = 0.0
        for bin_lower, bin_upper in zip(bin_lowers, bin_uppers):
            # determine if sample is in bin m (between bin lower & upper)
            in_bin = np.logical_and(confidences > bin_lower.item(), confidences <= bin_upper.item())
            # can calculate the empirical probability of a sample falling into bin m: (|Bm|/n)
            prob_in_bin = np.mean(in_bin)

            if prob_in_bin > 0:
                # get the accuracy of bin m: acc(Bm)
                avg_pred = np.mean(predictions[in_bin])
                # get the average confidence of bin m: conf(Bm)
                avg_confidence_in_bin = np.mean(confidences[in_bin])
                # calculate |acc(Bm) - conf(Bm)| * (|Bm|/n) for bin m and add to the total ECE
                ece += np.abs(avg_pred - avg_confidence_in_bin) * prob_in_bin
        return ece
    
    def evaluate(self, X, y):
        """
        Avalia o desempenho do classificador nas instâncias em X com os rótulos verdadeiros y.
        """

        results = {}
        y_prob = clf.predict_proba(X)
        predict_set = clf.predict_set(X)
        n_digits = 3
        results["one_c"] = round(np.sum([np.sum(p) == 1 for p in predict_set]) / len(y), n_digits)
        results["avg_c"] = round(np.sum(predict_set) / len(predict_set), n_digits)
        results["empty"] = round(np.sum([np.sum(p) == 0 for p in predict_set]) / len(y), n_digits)
        results["error"] = round(1-np.sum(predict_set[np.arange(len(y)), y]) / len(y), n_digits)
        results["log_loss"] = round(log_loss(y, y_prob[:,1]), n_digits)
        results["brier_loss"] = round(brier_score_loss(y, y_prob[:,1]), n_digits)
        results["ece"] = round(self.expected_calibration_error(y, y_prob), n_digits)

        return results

    

In [186]:
clf = WrapperOOBConformalClassifier(rf).fit(y_train)

/home/lucasleao/miniconda3/envs/autorizai/lib/python3.10/site-packages/venn_abers/venn_abers.py:102: RuntimeWarning:

All-NaN slice encountered



In [187]:
clf.evaluate(X_test, y_test)

{'one_c': 0.979,
 'avg_c': 0.979,
 'empty': 0.021,
 'error': 0.037,
 'log_loss': 0.088,
 'brier_loss': 0.02,
 'ece': 0.004}